In [33]:
%matplotlib inline
import pandas as pd

In [34]:
tmap = pd.read_csv('../data/Tmap_2005.csv')

In [35]:
tmap

,일자,지역(시도),지역(시군구),검색지명,검색지유형1,검색지유형2,검색지유형3,검색지랭킹(시군구내)
0,20200510,강원,삼척시,쏠비치삼척,여행/레저,숙박,콘도/리조트,1
1,20200510,강원,삼척시,삼척해양레일바이크용화정거장,여행/레저,레저/스포츠,레저/스포츠기타,2
2,20200510,강원,삼척시,장호항,교통편의,교통시설,항구,3
3,20200510,강원,삼척시,삼척해양레일바이크궁촌정거장,여행/레저,레저/스포츠,레저/스포츠기타,4
4,20200510,강원,삼척시,해신당공원,여행/레저,관광명소,공원,5
...,...,...,...,...,...,...,...,...
232495,20200503,충북,청주시 청원구,초정약수세종스파텔,여행/레저,관광명소,온천,26
232496,20200503,충북,청주시 청원구,더휴센트럴파크아파트,건물/시설,주요건물,아파트단지,27
232497,20200503,충북,청주시 청원구,오창대성베르힐아파트,건물/시설,주요건물,아파트단지,28
232498,20200503,충북,청주시 청원구,오창모아미래도와이드시티아파트,건물/시설,주요건물,아파트단지,29


In [36]:
tmap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232500 entries, 0 to 232499
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   일자           232500 non-null  int64 
 1   지역(시도)       232500 non-null  object
 2   지역(시군구)      232500 non-null  object
 3   검색지명         232500 non-null  object
 4   검색지유형1       232500 non-null  object
 5   검색지유형2       232500 non-null  object
 6   검색지유형3       232500 non-null  object
 7   검색지랭킹(시군구내)  232500 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 14.2+ MB


# 서울지역만 하는 경우
## 노원구 한정

In [61]:
c1 = tmap['지역(시도)'] == '서울'
c2 = tmap['지역(시군구)'] == '노원구'
# c2 = tmap['일자']==20200510
seoul = tmap.loc[c1 & c2,:]
seoul

,일자,지역(시도),지역(시군구),검색지명,검색지유형1,검색지유형2,검색지유형3,검색지랭킹(시군구내)
5880,20200528,서울,노원구,이마트월계점,쇼핑,대형유통점,할인점,1
5881,20200528,서울,노원구,이마트트레이더스월계점,쇼핑,대형유통점,할인점,2
5882,20200528,서울,노원구,롯데백화점노원점,쇼핑,대형유통점,백화점,3
5883,20200528,서울,노원구,롯데마트중계점,쇼핑,대형유통점,할인점,4
5884,20200528,서울,노원구,노원구청,공공편의,행정기관,도/시/구/군청,5
...,...,...,...,...,...,...,...,...
229645,20200527,서울,노원구,보람1단지아파트,건물/시설,주요건물,아파트단지,26
229646,20200527,서울,노원구,상계주공9단지아파트,건물/시설,주요건물,아파트건설사,27
229647,20200527,서울,노원구,세이브존노원점,쇼핑,대형유통점,할인점,28
229648,20200527,서울,노원구,중계무지개아파트,건물/시설,주요건물,아파트단지,29


In [62]:
cnt_seoul = seoul['검색지명'].value_counts()
cnt_seoul

공릉역             31
중계그린아파트         31
석계역             31
롯데백화점노원점        31
노원구청            31
상계주공7단지아파트      31
수락산역            31
태릉입구역           31
월계한진한화그랑빌아파트    31
당고개역            31
노원역             31
극동건영벽산아파트       31
이마트트레이더스월계점     31
이마트월계점          31
롯데마트중계점         31
상계역             31
상계주공10단지아파트     30
인제대부속상계백병원      28
상계주공15단지아파트     28
상계주공2단지아파트      26
태강아파트           26
학여울청구아파트        25
원자력병원           24
상계주공6단지아파트      23
중계주공5단지아파트      22
을지병원            22
중계무지개아파트        20
2001아울렛중계점      19
보람1단지아파트        17
태릉CC            16
상계주공14단지아파트     14
삼육대학교           12
광운대학교           12
하계장미6단지아파트      12
월계주공2단지아파트      12
공릉풍림아파트         10
원자력병원장례식장        7
상계주공3단지아파트       7
세이브존노원점          6
상계주공1단지아파트       6
건영백화점            3
상계주공9단지아파트       2
양지대림1차아파트        1
더블레스컨벤션          1
을지병원장례식장         1
인덕대학교            1
서울노원경찰서          1
Name: 검색지명, dtype: int64

In [66]:
type(cnt_seoul)
data = {'name':[], 'count':[]}
for idx, i in enumerate(cnt_seoul.index):
    if cnt_seoul[idx]<=1: break
    data['name'].append(i)
    data['count'].append(cnt_seoul[idx])
data
cnt_df = pd.DataFrame(data)
cnt_df.tail()
# 음 한달동안 서울에서 20회 이하는 삭제했을때 

,name,count
37,상계주공3단지아파트,7
38,세이브존노원점,6
39,상계주공1단지아파트,6
40,건영백화점,3
41,상계주공9단지아파트,2


# 카카오 api
- 카카오 api 키워드로 장소검색을 사용하면 각각의 네비게이션 목적지들의 좌표값이 나온다.
- 

In [74]:
# cnt_df.plot(kind='bar')
import requests
from decouple import config
ka_key = config('KAKAO_RESTAPI')
kakao_url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query=천호역'
h = {
    'Authorization': 'KakaoAK '+ka_key
}
res = requests.get(kakao_url, headers=h)
res.json()

{'documents': [{'address_name': '서울 강동구 성내동 62-1',
   'category_group_code': 'SW8',
   'category_group_name': '지하철역',
   'category_name': '교통,수송 > 지하철,전철 > 수도권5호선',
   'distance': '',
   'id': '8099936',
   'phone': '02-6311-5471',
   'place_name': '천호역 5호선',
   'place_url': 'http://place.map.kakao.com/8099936',
   'road_address_name': '서울 강동구 천호대로 지하 997',
   'x': '127.12392845044',
   'y': '37.5385112120297'},
  {'address_name': '서울 강동구 성내동 62-1',
   'category_group_code': 'SW8',
   'category_group_name': '지하철역',
   'category_name': '교통,수송 > 지하철,전철 > 수도권8호선',
   'distance': '',
   'id': '21160713',
   'phone': '02-6311-5471',
   'place_name': '천호역 8호선',
   'place_url': 'http://place.map.kakao.com/21160713',
   'road_address_name': '',
   'x': '127.123203357732',
   'y': '37.5379353309085'},
  {'address_name': '서울 강동구 성내동 62-1',
   'category_group_code': 'PK6',
   'category_group_name': '주차장',
   'category_name': '교통,수송 > 교통시설 > 주차장 > 공영주차장',
   'distance': '',
   'id': '961348435',
 

# 전체지역

In [43]:
cnt_tmap = tmap['검색지명'].value_counts()
cnt_tmap

솔내음                 95
하우스베이커리             93
목월빵집                93
이성당                 93
수림정                 93
                    ..
소담길                  1
용인행정타운두산위브2단지아파트     1
단촌역                  1
남이면사무소               1
영월드론전용비행시험장          1
Name: 검색지명, Length: 16307, dtype: int64